In [ ]:
# SPLIT DATE_TIME FUNCTIONS

# Assuming df_final_web_data contains the merged dataframe with a datetime column named 'timestamp'

# Convert 'timestamp' column to datetime if it's not already in datetime format
df_final_web_data['date_time'] = pd.to_datetime(df_final_web_data['date_time'])

# Split 'timestamp' into separate 'date' and 'time' columns
df_final_web_data['date'] = df_final_web_data['date_time'].dt.date
df_final_web_data['time'] = df_final_web_data['date_time'].dt.time

# Display the dataframe with the new columns
display(df_final_web_data.head(20))


In [ ]:
# 4.2 TIME SPENT ON EACH STEP --> with group

# Convert 'timestamp' column to datetime if it's not already in datetime format
df_final_web_data['date_time'] = pd.to_datetime(df_final_web_data['date_time'])

# 1. Filter Data for Control and Test Groups
control_data = df_final_web_data[df_final_web_data['Variation'] == 'Control']
test_data = df_final_web_data[df_final_web_data['Variation'] == 'Test']

# 2. Calculate Time Differences Between Consecutive Steps
control_data['time_diff'] = control_data.groupby('client_id')['date_time'].diff()
test_data['time_diff'] = test_data.groupby('client_id')['date_time'].diff()

# 3. Aggregate Time Differences for Each Step
control_average_time_per_step = control_data.groupby('process_step')['time_diff'].mean()
test_average_time_per_step = test_data.groupby('process_step')['time_diff'].mean()

# Print or visualize the results
print("Control Group:")
print(control_average_time_per_step)

print("\nTest Group:")
print(test_average_time_per_step)

In [ ]:
# 4.2 TIME SPENT ON EACH STEP --> without group

# Convert 'date_time' column to datetime if it's not already in datetime format
df_final_web_data['date_time'] = pd.to_datetime(df_final_web_data['date_time'])

# Sort the dataframe by client_id and date_time
df_final_web_data.sort_values(by=['client_id', 'date_time'], inplace=True)

# Calculate time differences between consecutive steps for each visit
df_final_web_data['time_diff'] = df_final_web_data.groupby('client_id')['date_time'].diff()

# Group by step and calculate the average time difference for each step
average_time_per_step = df_final_web_data.groupby('process_step')['time_diff'].mean()

# Print or visualize the results
display(average_time_per_step)

In [ ]:
# 3.1 COMPLETION RATE --> without unique_ids 
# 3.1.1 Calculate completion rate 

# Sort by client_id and date_time
df_final_web_data = df_final_web_data.sort_values(by=['client_id', 'date_time'])

# Function to calculate the completion rate
def calculate_completion_rate(df_web_data, completion_step='confirm'):
    # Ensure date_time is in datetime format
    df_final_web_data['date_time'] = pd.to_datetime(df_final_web_data['date_time'])
    
    # Identify clients who completed the process
    completed_clients = df_final_web_data[df_final_web_data['process_step'] == completion_step]['client_id'].unique()
    
    # Mark clients who completed the process
    df_final_web_data['confirm'] = df_final_web_data['client_id'].isin(completed_clients)
    
    # Group by variation to count total and completed clients
    completion_counts = df_final_web_data.groupby('Variation')['confirm'].agg(['sum', 'count'])
    completion_counts.columns = ['completed_clients', 'total_clients']
    
    # Calculate completion rate
    completion_counts['completion_rate'] = completion_counts['completed_clients'] / completion_counts['total_clients']
    
    return completion_counts

# Calculate the completion rate
completion_rate_df = calculate_completion_rate(df_final_web_data)

display(completion_rate_df)

# 50500 unique id with variation information --> some of them CONFIRM MORE THAN ONCE 

In [ ]:
# 3.1 COMPLETION RATE --> without unique_ids 
# 3.1.1 Calculate completion rate 

# Sort by client_id and date_time
df_final_web_data = df_final_web_data.sort_values(by=['client_id', 'date_time'])

# Function to calculate the completion rate
def calculate_completion_rate(df_web_data, completion_step='confirm'):
    # Ensure date_time is in datetime format
    df_final_web_data['date_time'] = pd.to_datetime(df_final_web_data['date_time'])
    
    # Identify clients who completed the process
    completed_clients = df_final_web_data[df_final_web_data['process_step'] == completion_step]['client_id'].unique()
    
    # Mark clients who completed the process
    df_final_web_data['confirm'] = df_final_web_data['client_id'].isin(completed_clients)
    
    # Group by variation to count total and completed clients
    completion_counts = df_final_web_data.groupby('Variation')['confirm'].agg(['sum', 'count'])
    completion_counts.columns = ['completed_clients', 'total_clients']
    
    # Calculate completion rate
    completion_counts['completion_rate'] = completion_counts['completed_clients'] / completion_counts['total_clients']
    
    return completion_counts

# Calculate the completion rate
completion_rate_df = calculate_completion_rate(df_final_web_data)

display(completion_rate_df)

# 50500 unique id with variation information --> some of them CONFIRM MORE THAN ONCE 

In [ ]:
import pandas as pd
from scipy.stats import ttest_ind

# Sample data (replace with actual data loading)
# df_final_web_data = pd.read_csv('path_to_your_web_data.csv')
# df_experiment_client = pd.read_csv('path_to_your_experiment_client.csv')

# Ensure 'date_time' is in datetime format
df_final_web_data['date_time'] = pd.to_datetime(df_final_web_data['date_time'])

# Filter Data for Control and Test Groups
control_data = df_final_web_data[df_final_web_data['Variation'] == 'Control']
test_data = df_final_web_data[df_final_web_data['Variation'] == 'Test']

# Calculate Time Differences Between Consecutive Steps
control_data['time_diff'] = control_data.groupby('client_id')['date_time'].diff()
test_data['time_diff'] = test_data.groupby('client_id')['date_time'].diff()

# Remove NaN values resulting from the diff() operation
control_data = control_data.dropna(subset=['time_diff'])
test_data = test_data.dropna(subset=['time_diff'])

# Initialize lists to store results
steps = []
t_statistics = []
p_values = []

# Perform a two-sample t-test for each step
for step in control_data['process_step'].unique():
    control_step_time = control_data[control_data['process_step'] == step]['time_diff'].dt.total_seconds()
    test_step_time = test_data[test_data['process_step'] == step]['time_diff'].dt.total_seconds()
    
    # Perform t-test
    t_stat, p_val = ttest_ind(control_step_time, test_step_time, equal_var=False)
    
    # Store results
    steps.append(step)
    t_statistics.append(t_stat)
    p_values.append(p_val)

# Create a DataFrame to display the results
results_df = pd.DataFrame({
    'Step': steps,
    'T-Statistic': t_statistics,
    'P-Value': p_values
})

# Print or visualize the results
print(results_df)

# Check if p-value is less than alpha (e.g., 0.05) for significance
alpha = 0.05
results_df['Significant'] = results_df['P-Value'] < alpha

print("\nHypothesis Testing Results:")
print(results_df)


In [ ]:
# 3.2 TIME SPENT ON EACH STEP
# 3.2.1 Calculate time spent on each step 

# Filter Data for Control and Test Groups
control_data = df_final_web_data[df_final_web_data['Variation'] == 'Control']
test_data = df_final_web_data[df_final_web_data['Variation'] == 'Test']

# Calculate Time Differences Between Consecutive Steps
control_data['time_diff'] = control_data.groupby('client_id')['date_time'].diff()
test_data['time_diff'] = test_data.groupby('client_id')['date_time'].diff()

# Aggregate Time Differences for Each Step
control_average_time_per_step = control_data.groupby('process_step')['time_diff'].mean()
test_average_time_per_step = test_data.groupby('process_step')['time_diff'].mean()

# Print or visualize the results
print("Control Group:")
print(control_average_time_per_step)

print("\nTest Group:")
print(test_average_time_per_step)


# 3.2.2 Set the hypothesis for Hypothis 2 --> time spent on each step 
# H0: There is no significant difference in time spent on each step between the Test and Control groups.
# H1: There is significant difference in time spent on each step between the Test and Control groups.

#BY USING TWO SAMPLE T-TEST 

from scipy.stats import ttest_ind

# Extract time differences for each step for both groups
control_time_diff = control_data.groupby('process_step')['time_diff'].apply(list)
test_time_diff = test_data.groupby('process_step')['time_diff'].apply(list)

# Perform the t-test for each step
t_test_results = {}
for step in control_time_diff.index.intersection(test_time_diff.index):
    t_stat, p_value = ttest_ind(control_time_diff[step].dropna(), test_time_diff[step].dropna())
    t_test_results[step] = {'t_stat': t_stat, 'p_value': p_value}

# Print the t-test results
for step, results in t_test_results.items():
    print(f"Step: {step}")
    print(f"  T-statistic: {results['t_stat']}")
    print(f"  P-value: {results['p_value']}")

In [ ]:
# 3.2.2 Set the hypothesis for Hypothis 2 --> time spent on each step: the mean value of time on each step: vertical: --> compare steps/5 of group/test 
# H0: The average time on each step of Test group is greater than Control group increased by 5%  
# H1: The average time on each step of Test group is equal to or less than Control group increased by 5%  

import pandas as pd
from scipy.stats import ttest_ind

# Filter Data for Control and Test Groups
control_data = df_final_web_data[df_final_web_data['Variation'] == 'Control']
test_data = df_final_web_data[df_final_web_data['Variation'] == 'Test']

# Calculate Time Differences Between Consecutive Steps
control_data['time_diff'] = control_data.groupby('client_id')['date_time'].diff()
test_data['time_diff'] = test_data.groupby('client_id')['date_time'].diff()

# Remove NaN values resulting from the diff() operation
control_data = control_data.dropna(subset=['time_diff'])
test_data = test_data.dropna(subset=['time_diff'])

# Initialize lists to store results
steps = []
t_statistics = []
p_values = []

# Perform a one-sided t-test for each step
for step in control_data['process_step'].unique():
    control_step_time = control_data[control_data['process_step'] == step]['time_diff'].dt.total_seconds()
    test_step_time = test_data[test_data['process_step'] == step]['time_diff'].dt.total_seconds()
    
    # Adjust control group mean by increasing it by 5%
    control_mean_adjusted = control_step_time.mean() * 1.05
    
    # Perform one-sided t-test (test if test group mean is less than or equal to adjusted control group mean)
    t_stat, p_val = ttest_ind(test_step_time, control_step_time, equal_var=False)
    
    # Since we want a one-sided test, we divide the p-value by 2
    if t_stat < 0:
        p_val = p_val / 2
    else:
        p_val = 1 - (p_val / 2)
    
    # Store results
    steps.append(step)
    t_statistics.append(t_stat)
    p_values.append(p_val)

# Create a DataFrame to display the results
results_df = pd.DataFrame({
    'Step': steps,
    'T-Statistic': t_statistics,
    'P-Value': p_values
})

# Print or visualize the results
display(results_df)

# Check if p-value is less than alpha (e.g., 0.05) for significance
alpha = 0.05
results_df['Significant'] = results_df['P-Value'] < alpha

print("\nHypothesis Testing Results:")
display(results_df)


In [ ]:
# 2.4 Replace missing value 
def replace_missing_value(df_final_web_data):    
    for column in  df_final_web_data.select_dtypes(include=[object]):
         df_final_web_data [column].fillna("Unknown", inplace=True)
    
    display( df_final_web_data )

    return  df_final_web_data

df_final_web_data = replace_missing_value(df_final_web_data)

In [ ]:
# BUGS??? Why tenure years > age? 
import pandas as pd

# Ensure the columns for age and tenure are in the correct data type
df_new2['clnt_age'] = pd.to_numeric(df_new2['clnt_age'], errors='coerce')
df_new2['clnt_tenure_yr'] = pd.to_numeric(df_new2['clnt_tenure_yr'], errors='coerce')

# Calculate the difference between age and tenure
df_new2['age_tenure_diff'] = df_new2['clnt_age'] - df_new2['clnt_tenure_yr']

# Filter rows where the difference is negative
negative_diff =df_new2[df_new2['age_tenure_diff'] < 0]

# Select client IDs
negative_diff_client_ids = negative_diff['client_id']

# Display the client IDs
display(negative_diff_client_ids)
